In [1]:
from pyspark.sql import SparkSession

# Spark 세션 초기화
spark = SparkSession.builder.appName("Ironman Data Analysis_241219_01").getOrCreate()

# CSV 다시 불러오기
file_path = "file:///home/lab12/src/data/ironman_wc_2022.csv"  # 정확한 경로 입력
df = spark.read.csv(file_path, header=True, inferSchema=True)

# 데이터 확인
df.show(truncate=False)
df.printSchema()

24/12/19 10:15:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+---+--------------------+--------------+------+----+--------+------------+------------+---------+---------+---------+---------+--------+--------+-------------+
|bib|name                |country       |gender|div |div_rank|overall_time|overall_rank|swim_time|swim_rank|bike_time|bike_rank|run_time|run_rank|finish_status|
+---+--------------------+--------------+------+----+--------+------------+------------+---------+---------+---------+---------+--------+--------+-------------+
|8  |Gustav Iden         |Norway        |Male  |MPRO|1       |7:40:24     |1           |48:23:00 |10       |4:11:06  |6        |2:36:15 |1       |Finisher     |
|15 |Sam Laidlow         |France        |Male  |MPRO|2       |7:42:24     |2           |48:16:00 |2        |4:04:36  |1        |2:44:40 |5       |Finisher     |
|1  |Kristian Blummenfelt|Norway        |Male  |MPRO|3       |7:43:23     |3           |48:20:00 |5        |4:11:16  |8        |2:39:21 |2       |Finisher     |
|23 |Max Neumann         |Australi

In [4]:
from pyspark.sql.functions import when, col, split

# 1. 시간 데이터를 초 단위로 변환하는 함수 정의
def time_to_seconds(time_str_col):
    """
    문자열 형태의 시간 데이터를 초 단위로 변환
    """
    return (split(time_str_col, ":")[0].cast("int") * 3600 +
            split(time_str_col, ":")[1].cast("int") * 60 +
            split(time_str_col, ":")[2].cast("int"))

# 2. 초 단위 컬럼 생성
df = df.withColumn("swim_seconds", time_to_seconds(col("swim_time"))) \
       .withColumn("bike_seconds", time_to_seconds(col("bike_time"))) \
       .withColumn("run_seconds", time_to_seconds(col("run_time"))) \
       .withColumn("overall_seconds", time_to_seconds(col("overall_time")))

# 3. 컷오프 기준에 따른 DNF 컬럼 생성
df = df.withColumn(
    "DNF",
    when((col("swim_seconds") > 2 * 3600 + 20 * 60) |  # 수영 컷오프
         (col("swim_seconds") + col("bike_seconds") > 10 * 3600 + 30 * 60) |  # 수영 + 사이클 컷오프
         (col("overall_seconds") > 17 * 3600), 1).otherwise(0)  # 전체 컷오프
)

# 4. 결과 확인
df.select("swim_seconds", "bike_seconds", "run_seconds", "overall_seconds", "DNF").show()


+------------+------------+-----------+---------------+---+
|swim_seconds|bike_seconds|run_seconds|overall_seconds|DNF|
+------------+------------+-----------+---------------+---+
|      174180|       15066|       9375|          27624|  1|
|      173760|       14676|       9880|          27744|  1|
|      174000|       15076|       9561|          27803|  1|
|      174300|       15090|       9614|          27884|  1|
|      190500|       15071|       9926|          28445|  1|
|      190680|       14951|      10125|          28540|  1|
|      190440|       14945|      10168|          28552|  1|
|      179340|       15218|      10091|          28598|  1|
|      179400|       15314|       9960|          28618|  1|
|      178920|       15712|       9719|          28700|  1|
|      190260|       14944|      10467|          28851|  1|
|      173700|       15478|      10229|          28913|  1|
|      174180|       15210|      10563|          28978|  1|
|      174360|       15779|      10023| 

In [6]:
df = df.filter((col("finish_status") != "DNS") & (col("finish_status") != "DQ"))


In [7]:
df = df.dropna(subset=["swim_seconds", "bike_seconds", "run_seconds", "overall_seconds"])


In [8]:
df = df.withColumn("swim_seconds", col("swim_seconds").cast("double")) \
       .withColumn("bike_seconds", col("bike_seconds").cast("double")) \
       .withColumn("run_seconds", col("run_seconds").cast("double")) \
       .withColumn("overall_seconds", col("overall_seconds").cast("double"))


In [9]:
df.groupBy("DNF").count().show()


+---+-----+
|DNF|count|
+---+-----+
|  1|  345|
|  0| 2031|
+---+-----+



In [12]:
df_finishers = df.filter(col("DNF") == 0)
df_finishers.select("DNF").groupBy("DNF").count().show()

+---+-----+
|DNF|count|
+---+-----+
|  0| 2031|
+---+-----+



In [14]:
from pyspark.ml.feature import StringIndexer

# gender 컬럼 인코딩
indexer = StringIndexer(inputCol="gender", outputCol="gender_encoded")
df_finishers = indexer.fit(df_finishers).transform(df_finishers)

# 결과 확인
df_finishers.select("gender", "gender_encoded").show(5)


+------+--------------+
|gender|gender_encoded|
+------+--------------+
|  Male|           0.0|
|  Male|           0.0|
|  Male|           0.0|
|  Male|           0.0|
|  Male|           0.0|
+------+--------------+
only showing top 5 rows



In [16]:
from pyspark.sql.functions import when, col

# div 컬럼을 기반으로 age_group 생성
df_finishers = df_finishers.withColumn(
    "age_group",
    when(col("div").startswith("M18-24"), 18)
    .when(col("div").startswith("M25-29"), 25)
    .when(col("div").startswith("M30-34"), 30)
    .when(col("div").startswith("M35-39"), 35)
    .when(col("div").startswith("M40-44"), 40)
    .when(col("div").startswith("M45-49"), 45)
    .when(col("div").startswith("M50-54"), 50)
    .when(col("div").startswith("M55-59"), 55)
    .when(col("div").startswith("MPRO"), 0)  # 프로 선수
    .otherwise(None)  # 나머지 경우 처리
)

# 결과 확인
df_finishers.select("div", "age_group").distinct().show()


+------+---------+
|   div|age_group|
+------+---------+
|M30-34|       30|
|M25-29|       25|
|M18-24|       18|
|M35-39|       35|
|M40-44|       40|
|M55-59|       55|
|M45-49|       45|
|M50-54|       50|
+------+---------+



In [17]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import when, col

# 1. 순위 그룹 라벨링
total_participants = df_finishers.count()
top_10 = total_participants * 0.1
top_25 = total_participants * 0.25
top_50 = total_participants * 0.5

df_finishers = df_finishers.withColumn(
    "rank_range",
    when(col("overall_rank") <= top_10, "Top 10%")
    .when((col("overall_rank") > top_10) & (col("overall_rank") <= top_25), "Top 25%")
    .when((col("overall_rank") > top_25) & (col("overall_rank") <= top_50), "Top 50%")
    .otherwise("Bottom 50%")
)

# 2. 피처 벡터 생성
assembler = VectorAssembler(
    inputCols=["gender_encoded", "age_group", "swim_seconds", "bike_seconds", "run_seconds"],
    outputCol="features"
)

df_final = assembler.transform(df_finishers).select("features", "rank_range")

# 결과 확인
df_final.show(5, truncate=False)


+---------------------------------+----------+
|features                         |rank_range|
+---------------------------------+----------+
|[0.0,30.0,3655.0,16953.0,11146.0]|Top 10%   |
|[0.0,30.0,3983.0,17318.0,10560.0]|Top 10%   |
|[0.0,30.0,3755.0,17022.0,11094.0]|Top 10%   |
|[0.0,35.0,4042.0,16196.0,11461.0]|Top 10%   |
|[0.0,30.0,3955.0,17532.0,10395.0]|Top 10%   |
+---------------------------------+----------+
only showing top 5 rows



In [18]:
# 데이터 분할
train_data, test_data = df_final.randomSplit([0.8, 0.2], seed=42)

# 데이터 크기 확인
print("훈련 데이터 크기:", train_data.count())
print("테스트 데이터 크기:", test_data.count())


훈련 데이터 크기: 1671
테스트 데이터 크기: 360


In [33]:
# train_data 스키마 및 데이터 확인
train_data.printSchema()
train_data.show(5, truncate=False)

# test_data 스키마 및 데이터 확인
test_data.printSchema()
test_data.show(5, truncate=False)


root
 |-- features: vector (nullable = true)
 |-- rank_range: string (nullable = false)

+---------------------------------+----------+
|features                         |rank_range|
+---------------------------------+----------+
|[0.0,18.0,3653.0,20368.0,12874.0]|Top 50%   |
|[0.0,18.0,3693.0,19738.0,17137.0]|Bottom 50%|
|[0.0,18.0,3722.0,19532.0,17446.0]|Bottom 50%|
|[0.0,18.0,3729.0,18523.0,14229.0]|Top 50%   |
|[0.0,18.0,3732.0,18024.0,16481.0]|Bottom 50%|
+---------------------------------+----------+
only showing top 5 rows

root
 |-- features: vector (nullable = true)
 |-- rank_range: string (nullable = false)

+---------------------------------+----------+
|features                         |rank_range|
+---------------------------------+----------+
|[0.0,18.0,3717.0,19283.0,13253.0]|Top 50%   |
|[0.0,18.0,3776.0,18633.0,12743.0]|Top 50%   |
|[0.0,18.0,3855.0,19772.0,14458.0]|Bottom 50%|
|[0.0,18.0,3943.0,18864.0,14091.0]|Top 50%   |
|[0.0,18.0,4134.0,18740.0,17225.0]|Bottom 50%

In [35]:
from pyspark.sql.functions import col

# 기존 rank_range_index가 있다면 삭제
if "rank_range_index" in df_final.columns:
    df_final = df_final.drop("rank_range_index")

# rank_range를 숫자형 rank_range_index로 변환
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="rank_range", outputCol="rank_range_index")
df_final = indexer.fit(df_final).transform(df_final)

# 결과 확인
df_final.select("rank_range", "rank_range_index").distinct().show()


+----------+----------------+
|rank_range|rank_range_index|
+----------+----------------+
|   Top 10%|             3.0|
|   Top 25%|             2.0|
|Bottom 50%|             0.0|
|   Top 50%|             1.0|
+----------+----------------+



In [36]:
# 데이터 분할
train_data, test_data = df_final.randomSplit([0.8, 0.2], seed=42)

# 데이터 확인
train_data.printSchema()
train_data.show(5, truncate=False)


root
 |-- features: vector (nullable = true)
 |-- rank_range: string (nullable = false)
 |-- rank_range_index: double (nullable = false)

+---------------------------------+----------+----------------+
|features                         |rank_range|rank_range_index|
+---------------------------------+----------+----------------+
|[0.0,18.0,3653.0,20368.0,12874.0]|Top 50%   |1.0             |
|[0.0,18.0,3693.0,19738.0,17137.0]|Bottom 50%|0.0             |
|[0.0,18.0,3722.0,19532.0,17446.0]|Bottom 50%|0.0             |
|[0.0,18.0,3729.0,18523.0,14229.0]|Top 50%   |1.0             |
|[0.0,18.0,3732.0,18024.0,16481.0]|Bottom 50%|0.0             |
+---------------------------------+----------+----------------+
only showing top 5 rows



In [37]:
# 데이터 분할
train_data, test_data = df_final.randomSplit([0.8, 0.2], seed=42)

# 데이터 크기 확인
print("훈련 데이터 크기:", train_data.count())
print("테스트 데이터 크기:", test_data.count())

# 데이터 샘플 확인
train_data.show(5, truncate=False)


훈련 데이터 크기: 1671
테스트 데이터 크기: 360
+---------------------------------+----------+----------------+
|features                         |rank_range|rank_range_index|
+---------------------------------+----------+----------------+
|[0.0,18.0,3653.0,20368.0,12874.0]|Top 50%   |1.0             |
|[0.0,18.0,3693.0,19738.0,17137.0]|Bottom 50%|0.0             |
|[0.0,18.0,3722.0,19532.0,17446.0]|Bottom 50%|0.0             |
|[0.0,18.0,3729.0,18523.0,14229.0]|Top 50%   |1.0             |
|[0.0,18.0,3732.0,18024.0,16481.0]|Bottom 50%|0.0             |
+---------------------------------+----------+----------------+
only showing top 5 rows



In [72]:
from pyspark.ml.classification import RandomForestClassifier

# Random Forest 모델 생성 및 학습
rf = RandomForestClassifier(labelCol="rank_range_index", featuresCol="features", numTrees=50)
rf_model = rf.fit(train_data)

# 테스트 데이터로 예측
predictions = rf_model.transform(test_data)

# 결과 확인
predictions.select("features", "rank_range_index", "prediction", "probability").show(10, truncate=False)


+---------------------------------+----------------+----------+----------------------------------------------------------------------------------+
|features                         |rank_range_index|prediction|probability                                                                       |
+---------------------------------+----------------+----------+----------------------------------------------------------------------------------+
|[0.0,18.0,3717.0,19283.0,13253.0]|1.0             |1.0       |[0.19375110197035592,0.7694830196427098,0.03667198167331931,9.389671361502349E-5] |
|[0.0,18.0,3776.0,18633.0,12743.0]|1.0             |1.0       |[0.06460296006524288,0.5638580102696178,0.3352770956312607,0.03626193403387875]   |
|[0.0,18.0,3855.0,19772.0,14458.0]|0.0             |0.0       |[0.8717716701471933,0.1264885405067432,0.0017397893460634995,0.0]                 |
|[0.0,18.0,3943.0,18864.0,14091.0]|1.0             |1.0       |[0.3335679328557881,0.6208305351463654,0.04543486533117

In [73]:
# 학습 데이터 스키마 확인
train_data.printSchema()

# 학습 데이터 상위 5개 확인
train_data.show(5)


root
 |-- features: vector (nullable = true)
 |-- rank_range: string (nullable = false)
 |-- rank_range_index: double (nullable = false)

+--------------------+----------+----------------+
|            features|rank_range|rank_range_index|
+--------------------+----------+----------------+
|[0.0,18.0,3653.0,...|   Top 50%|             1.0|
|[0.0,18.0,3693.0,...|Bottom 50%|             0.0|
|[0.0,18.0,3722.0,...|Bottom 50%|             0.0|
|[0.0,18.0,3729.0,...|   Top 50%|             1.0|
|[0.0,18.0,3732.0,...|Bottom 50%|             0.0|
+--------------------+----------+----------------+
only showing top 5 rows



In [74]:
from pyspark.ml.classification import RandomForestClassifier

# Random Forest 모델 생성 및 학습
rf = RandomForestClassifier(labelCol="rank_range_index", featuresCol="features", numTrees=50)
rf_model = rf.fit(train_data)

# 테스트 데이터로 예측
predictions = rf_model.transform(test_data)

# 결과 확인
predictions.select("features", "rank_range_index", "prediction", "probability").show(10, truncate=False)


+---------------------------------+----------------+----------+----------------------------------------------------------------------------------+
|features                         |rank_range_index|prediction|probability                                                                       |
+---------------------------------+----------------+----------+----------------------------------------------------------------------------------+
|[0.0,18.0,3717.0,19283.0,13253.0]|1.0             |1.0       |[0.19375110197035592,0.7694830196427098,0.03667198167331931,9.389671361502349E-5] |
|[0.0,18.0,3776.0,18633.0,12743.0]|1.0             |1.0       |[0.06460296006524288,0.5638580102696178,0.3352770956312607,0.03626193403387875]   |
|[0.0,18.0,3855.0,19772.0,14458.0]|0.0             |0.0       |[0.8717716701471933,0.1264885405067432,0.0017397893460634995,0.0]                 |
|[0.0,18.0,3943.0,18864.0,14091.0]|1.0             |1.0       |[0.3335679328557881,0.6208305351463654,0.04543486533117

In [75]:
import pandas as pd

# Spark DataFrame -> Pandas DataFrame 변환
result_pd = predictions.select("rank_range_index", "prediction", "probability").toPandas()

# 클래스 매핑 (숫자 -> 범위 이름)
class_mapping = {
    0.0: "Bottom 50%",
    1.0: "Top 50%",
    2.0: "Top 25%",
    3.0: "Top 10%"
}

# 매핑 적용
result_pd["Actual Rank"] = result_pd["rank_range_index"].map(class_mapping)
result_pd["Predicted Rank"] = result_pd["prediction"].map(class_mapping)

# 확률 정보 문자열로 포맷
result_pd["Probability"] = result_pd["probability"].apply(
    lambda x: f"Bottom 50%: {x[0]:.2%}, Top 50%: {x[1]:.2%}, Top 25%: {x[2]:.2%}, Top 10%: {x[3]:.2%}"
)

# 필요없는 컬럼 제거
result_pd = result_pd[["Actual Rank", "Predicted Rank", "Probability"]]

# 결과 출력
print(result_pd.head(10))


  Actual Rank Predicted Rank  \
0     Top 50%        Top 50%   
1     Top 50%        Top 50%   
2  Bottom 50%     Bottom 50%   
3     Top 50%        Top 50%   
4  Bottom 50%     Bottom 50%   
5     Top 50%        Top 50%   
6     Top 50%        Top 50%   
7  Bottom 50%     Bottom 50%   
8  Bottom 50%     Bottom 50%   
9     Top 50%        Top 50%   

                                         Probability  
0  Bottom 50%: 19.38%, Top 50%: 76.95%, Top 25%: ...  
1  Bottom 50%: 6.46%, Top 50%: 56.39%, Top 25%: 3...  
2  Bottom 50%: 87.18%, Top 50%: 12.65%, Top 25%: ...  
3  Bottom 50%: 33.36%, Top 50%: 62.08%, Top 25%: ...  
4  Bottom 50%: 99.23%, Top 50%: 0.77%, Top 25%: 0...  
5  Bottom 50%: 10.43%, Top 50%: 80.81%, Top 25%: ...  
6  Bottom 50%: 6.66%, Top 50%: 67.62%, Top 25%: 2...  
7  Bottom 50%: 96.44%, Top 50%: 3.56%, Top 25%: 0...  
8  Bottom 50%: 96.81%, Top 50%: 3.19%, Top 25%: 0...  
9  Bottom 50%: 4.23%, Top 50%: 60.46%, Top 25%: 3...  


In [76]:
# 사용자 입력 데이터 예시
input_data = [
    [1.0, 33.0, 7500.0, 16000.0, 14000.0],  # Gender, Age, Swim, Bike, Run
    [0.0, 28.0, 9000.0, 18000.0, 17000.0]   # Example 2
]

# 입력 데이터 변환
input_df = spark.createDataFrame(input_data, schema=["gender_encoded", "age_group", "swim_seconds", "bike_seconds", "run_seconds"])
input_features = assembler.transform(input_df)

# 예측 수행
input_predictions = rf_model.transform(input_features)

# 결과 출력 포맷
input_result = input_predictions.select("features", "prediction", "probability").toPandas()

# 클래스 매핑 적용
input_result["Predicted Rank"] = input_result["prediction"].map(class_mapping)
input_result["Probability"] = input_result["probability"].apply(
    lambda x: f"Bottom 50%: {x[0]:.2%}, Top 50%: {x[1]:.2%}, Top 25%: {x[2]:.2%}, Top 10%: {x[3]:.2%}"
)

# 최종 출력
print(input_result[["Predicted Rank", "Probability"]])


  Predicted Rank                                        Probability
0        Top 50%  Bottom 50%: 28.34%, Top 50%: 54.46%, Top 25%: ...
1     Bottom 50%  Bottom 50%: 97.48%, Top 50%: 2.52%, Top 25%: 0...


In [79]:
# 사용자 입력 데이터를 아이언맨 기준으로 변환하는 함수
def convert_to_ironman_seconds(swim_time, bike_time, run_time):
    """
    수영, 자전거, 달리기 기록을 입력받아 초 단위로 변환.
    """
    def time_to_seconds(time_str):
        h, m, s = map(int, time_str.split(":"))
        return h * 3600 + m * 60 + s

    swim_seconds = time_to_seconds(swim_time)
    bike_seconds = time_to_seconds(bike_time)
    run_seconds = time_to_seconds(run_time)
    return swim_seconds, bike_seconds, run_seconds


# 사용자 입력 데이터
user_data = {
    "gender": "남성",
    "age": 33,
    "swim_time": "00:26:48",  # 수영 1.5km
    "bike_time": "01:39:35",  # 자전거 40km
    "run_time": "00:51:31",   # 달리기 10km
}

# 입력 데이터를 변환
swim_seconds, bike_seconds, run_seconds = convert_to_ironman_seconds(
    user_data["swim_time"], user_data["bike_time"], user_data["run_time"]
)

# 아이언맨 기준 변환 (1.5km -> 3.8km, 40km -> 180km, 10km -> 42.2km)
swim_seconds = swim_seconds * (3.8 / 1.5)
bike_seconds = bike_seconds * (180 / 40)
run_seconds = run_seconds * (42.2 / 10)

# 모델 입력 데이터 구성
input_data = [[1.0, user_data["age"], swim_seconds, bike_seconds, run_seconds]]
input_df = spark.createDataFrame(input_data, schema=["gender_encoded", "age_group", "swim_seconds", "bike_seconds", "run_seconds"])

# 입력 데이터 변환 및 예측
input_features = assembler.transform(input_df)
predictions = rf_model.transform(input_features)

# 예측 결과 해석
prediction = predictions.select("prediction", "probability").collect()[0]
predicted_rank = class_mapping[prediction["prediction"]]
probabilities = prediction["probability"]

# 개선 방향 분석
improvement_guidelines = {
    "swim": "상위 10%에 진입하려면 1시간 5분 이하로 줄여야 합니다.",
    "bike": "상위 10%에 진입하려면 6시간 0분 이하로 줄여야 합니다.",
    "run": "현재 기록이 평균 이상이며 추가 개선은 필요하지 않습니다.",
}

# 출력 결과 구성
print("예상 데이터:")
print(f"- 성별: {user_data['gender']}")
print(f"- 나이: {user_data['age']}세")
print(f"- 예상기록:")
print(f"    - 수영 (3.8km): {swim_seconds // 3600:02.0f}:{(swim_seconds % 3600) // 60:02.0f}:{swim_seconds % 60:02.0f}")
print(f"    - 자전거 (180km): {bike_seconds // 3600:02.0f}:{(bike_seconds % 3600) // 60:02.0f}:{bike_seconds % 60:02.0f}")
print(f"    - 달리기 (42.2km): {run_seconds // 3600:02.0f}:{(run_seconds % 3600) // 60:02.0f}:{run_seconds % 60:02.0f}")
print("\n모델 예측 결과:")
print(f"- 예상 종합 순위 그룹: {predicted_rank}")
print(f"- 예상 부문 순위 그룹 (M30-34): {predicted_rank}")
print("\n종목별 개선 방향:")
for event, guideline in improvement_guidelines.items():
    print(f"    - {event.capitalize()}: {guideline}")


입력 데이터:
- 성별: 남성
- 나이: 33세
- 기록:
    - 수영 (3.8km): 01:07:54
    - 자전거 (180km): 07:28:08
    - 달리기 (42.2km): 03:37:24

모델 예측 결과:
- 예상 종합 순위 그룹: Bottom 50%
- 예상 부문 순위 그룹 (성별: 남성, 나이 그룹: 30대): Bottom 50%

종목별 개선 방향:
    - Swim: 상위 10%에 진입하려면 1시간 5분 이하로 줄여야 합니다.
    - Bike: 상위 10%에 진입하려면 6시간 0분 이하로 줄여야 합니다.
    - Run: 현재 기록이 평균 이상이며 추가 개선은 필요하지 않습니다.


In [80]:
# 사용자 데이터
user_data = {
    "gender": "남성",
    "age": 33,
    "swim_time": "00:26:48",  # 수영 1.5km
    "bike_time": "01:39:35",  # 자전거 40km
    "run_time": "00:51:31",   # 달리기 10km
}

# 성별 변환
gender_encoded = 1.0 if user_data["gender"] == "남성" else 0.0

# 시간 데이터를 초 단위로 변환 함수
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(":"))
    return h * 3600 + m * 60 + s

# 입력 데이터를 초 단위로 변환 및 거리 변환
swim_seconds = time_to_seconds(user_data["swim_time"]) * (3.8 / 1.5)  # 수영 3.8km 기준 변환
bike_seconds = time_to_seconds(user_data["bike_time"]) * (180 / 40)  # 자전거 180km 기준 변환
run_seconds = time_to_seconds(user_data["run_time"]) * (42.2 / 10)  # 달리기 42.2km 기준 변환

# 모델 입력 데이터 생성
input_data = [[gender_encoded, user_data["age"], swim_seconds, bike_seconds, run_seconds]]
input_df = spark.createDataFrame(input_data, schema=["gender_encoded", "age_group", "swim_seconds", "bike_seconds", "run_seconds"])
input_features = assembler.transform(input_df)

# 모델 예측
predictions = rf_model.transform(input_features)
prediction = predictions.select("prediction", "probability").collect()[0]
predicted_rank = class_mapping[prediction["prediction"]]
probabilities = prediction["probability"]

# 개선 방향 분석 (예제)
improvement_guidelines = {
    "swim": "상위 10%에 진입하려면 1시간 5분 이하로 줄여야 합니다.",
    "bike": "상위 10%에 진입하려면 6시간 0분 이하로 줄여야 합니다.",
    "run": "현재 기록이 평균 이상이며 추가 개선은 필요하지 않습니다.",
}

# 결과 출력
print("입력 데이터:")
print(f"- 성별: {user_data['gender']}")
print(f"- 나이: {user_data['age']}세")
print(f"- 기록:")
print(f"    - 수영 (3.8km): {swim_seconds // 3600:02.0f}:{(swim_seconds % 3600) // 60:02.0f}:{swim_seconds % 60:02.0f}")
print(f"    - 자전거 (180km): {bike_seconds // 3600:02.0f}:{(bike_seconds % 3600) // 60:02.0f}:{bike_seconds % 60:02.0f}")
print(f"    - 달리기 (42.2km): {run_seconds // 3600:02.0f}:{(run_seconds % 3600) // 60:02.0f}:{run_seconds % 60:02.0f}")
print("\n모델 예측 결과:")
print(f"- 예상 종합 순위 그룹: {predicted_rank}")
print(f"- 예상 부문 순위 그룹 (성별: {user_data['gender']}, 나이 그룹: {user_data['age'] // 10 * 10}대): {predicted_rank}")
print("\n종목별 개선 방향:")
for event, guideline in improvement_guidelines.items():
    print(f"    - {event.capitalize()}: {guideline}")


입력 데이터:
- 성별: 남성
- 나이: 33세
- 기록:
    - 수영 (3.8km): 01:07:54
    - 자전거 (180km): 07:28:08
    - 달리기 (42.2km): 03:37:24

모델 예측 결과:
- 예상 종합 순위 그룹: Bottom 50%
- 예상 부문 순위 그룹 (성별: 남성, 나이 그룹: 30대): Bottom 50%

종목별 개선 방향:
    - Swim: 상위 10%에 진입하려면 1시간 5분 이하로 줄여야 합니다.
    - Bike: 상위 10%에 진입하려면 6시간 0분 이하로 줄여야 합니다.
    - Run: 현재 기록이 평균 이상이며 추가 개선은 필요하지 않습니다.


In [82]:
# 필요한 모듈 임포트
from pyspark.ml.classification import RandomForestClassifier

# 시간 데이터를 초 단위로 변환하는 함수
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(":"))
    return h * 3600 + m * 60 + s

# 클래스 매핑
class_mapping = {
    0.0: "Bottom 50%",
    1.0: "Top 50%",
    2.0: "Top 25%",
    3.0: "Top 10%"
}

# 개선 방향 예시
improvement_guidelines = {
    "swim": "상위 10%에 진입하려면 1시간 5분 이하로 줄여야 합니다.",
    "bike": "상위 10%에 진입하려면 6시간 0분 이하로 줄여야 합니다.",
    "run": "현재 기록이 평균 이상이며 추가 개선은 필요하지 않습니다.",
}

# 사용자 입력 데이터 받기
print("아이언맨 기록을 입력해주세요:")
gender = input("성별 (남성/여성): ")
age = int(input("나이: "))
swim_time = input("수영 기록 (hh:mm:ss, 1.5km 기준): ")
bike_time = input("자전거 기록 (hh:mm:ss, 40km 기준): ")
run_time = input("달리기 기록 (hh:mm:ss, 10km 기준): ")

# 성별 인코딩 및 시간 변환
gender_encoded = 1.0 if gender == "남성" else 0.0
swim_seconds = time_to_seconds(swim_time) * (3.8 / 1.5)  # 수영 3.8km로 변환
bike_seconds = time_to_seconds(bike_time) * (180 / 40)  # 자전거 180km로 변환
run_seconds = time_to_seconds(run_time) * (42.2 / 10)  # 달리기 42.2km로 변환

# 입력 데이터 생성
input_data = [[gender_encoded, age, swim_seconds, bike_seconds, run_seconds]]
input_df = spark.createDataFrame(input_data, schema=["gender_encoded", "age_group", "swim_seconds", "bike_seconds", "run_seconds"])
input_features = assembler.transform(input_df)

# 모델 예측
predictions = rf_model.transform(input_features)
prediction = predictions.select("prediction", "probability").collect()[0]
predicted_rank = class_mapping[prediction["prediction"]]
probabilities = prediction["probability"]

# 결과 출력
print("\n모델 예측 결과:")
print(f"- 예상 종합 순위 그룹: {predicted_rank}")
print(f"- 예상 부문 순위 그룹 (성별: {gender}, 나이 그룹: {age // 10 * 10}대): {predicted_rank}")
print("\n종목별 개선 방향:")
for event, guideline in improvement_guidelines.items():
    print(f"    - {event.capitalize()}: {guideline}")


아이언맨 기록을 입력해주세요:
성별 (남성/여성): 여성
나이: 37
수영 기록 (hh:mm:ss, 1.5km 기준): 00:26:48
자전거 기록 (hh:mm:ss, 40km 기준): 01:39:35
달리기 기록 (hh:mm:ss, 10km 기준): 00:51:32

모델 예측 결과:
- 예상 종합 순위 그룹: Bottom 50%
- 예상 부문 순위 그룹 (성별: 여성, 나이 그룹: 30대): Bottom 50%

종목별 개선 방향:
    - Swim: 상위 10%에 진입하려면 1시간 5분 이하로 줄여야 합니다.
    - Bike: 상위 10%에 진입하려면 6시간 0분 이하로 줄여야 합니다.
    - Run: 현재 기록이 평균 이상이며 추가 개선은 필요하지 않습니다.


In [83]:
# 필요한 모듈 임포트
from pyspark.ml.classification import RandomForestClassifier

# 시간 데이터를 초 단위로 변환하는 함수
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(":"))
    return h * 3600 + m * 60 + s

# 초를 hh:mm:ss 형식으로 변환하는 함수
def seconds_to_time(seconds):
    h = int(seconds // 3600)
    m = int((seconds % 3600) // 60)
    s = int(seconds % 60)
    return f"{h:02}:{m:02}:{s:02}"

# 클래스 매핑
class_mapping = {
    0.0: "Bottom 50%",
    1.0: "Top 50%",
    2.0: "Top 25%",
    3.0: "Top 10%"
}

# 예상 기록 계산 함수
def calculate_average_times(predicted_group):
    avg_times = {
        "Bottom 50%": {"swim": 1.5 * 3600, "bike": 6.5 * 3600, "run": 4.5 * 3600},
        "Top 50%": {"swim": 1.4 * 3600, "bike": 6.0 * 3600, "run": 4.0 * 3600},
        "Top 25%": {"swim": 1.3 * 3600, "bike": 5.5 * 3600, "run": 3.5 * 3600},
        "Top 10%": {"swim": 1.2 * 3600, "bike": 5.0 * 3600, "run": 3.0 * 3600},
    }
    return avg_times[predicted_group]

# 사용자 입력 데이터 받기
print("자신의 기록을 입력해주세요:")
gender = input("성별 (남성/여성): ")
age = int(input("나이: "))
swim_time = input("수영 기록 (hh:mm:ss, 1.5km 기준): ")
bike_time = input("자전거 기록 (hh:mm:ss, 40km 기준): ")
run_time = input("달리기 기록 (hh:mm:ss, 10km 기준): ")

# 성별 인코딩 및 시간 변환
gender_encoded = 1.0 if gender == "남성" else 0.0
swim_seconds = time_to_seconds(swim_time) * (3.8 / 1.5)  # 수영 3.8km로 변환
bike_seconds = time_to_seconds(bike_time) * (180 / 40)  # 자전거 180km로 변환
run_seconds = time_to_seconds(run_time) * (42.2 / 10)  # 달리기 42.2km로 변환

# 입력 데이터 생성
input_data = [[gender_encoded, age, swim_seconds, bike_seconds, run_seconds]]
input_df = spark.createDataFrame(input_data, schema=["gender_encoded", "age_group", "swim_seconds", "bike_seconds", "run_seconds"])
input_features = assembler.transform(input_df)

# 모델 예측
predictions = rf_model.transform(input_features)
prediction = predictions.select("prediction", "probability").collect()[0]
predicted_rank = class_mapping[prediction["prediction"]]
probabilities = prediction["probability"]

# 예상 기록 계산
average_times = calculate_average_times(predicted_rank)

# 결과 출력
print("\n모델 예측 결과:")
print(f"- 예상 종합 순위 그룹: {predicted_rank}")
print(f"- 예상 부문 순위 그룹 (성별: {gender}, 나이 그룹: {age // 10 * 10}대): {predicted_rank}")

print("\n예상 기록:")
print(f"    - 수영: {seconds_to_time(average_times['swim'])}")
print(f"    - 자전거: {seconds_to_time(average_times['bike'])}")
print(f"    - 달리기: {seconds_to_time(average_times['run'])}")

print("\n종목별 개선 방향:")
print("    - 수영: 상위 10%에 진입하려면 1시간 5분 이하로 줄여야 합니다.")
print("    - 자전거: 상위 10%에 진입하려면 6시간 0분 이하로 줄여야 합니다.")
print("    - 달리기: 현재 기록이 평균 이상이며 추가 개선은 필요하지 않습니다.")


아이언맨 기록을 입력해주세요:
성별 (남성/여성): 여성
나이: 37
수영 기록 (hh:mm:ss, 1.5km 기준): 00:26:48
자전거 기록 (hh:mm:ss, 40km 기준): 01:39:35
달리기 기록 (hh:mm:ss, 10km 기준): 00:51:32

모델 예측 결과:
- 예상 종합 순위 그룹: Bottom 50%
- 예상 부문 순위 그룹 (성별: 여성, 나이 그룹: 30대): Bottom 50%

예상 기록:
    - 수영: 01:30:00
    - 자전거: 06:30:00
    - 달리기: 04:30:00

종목별 개선 방향:
    - 수영: 상위 10%에 진입하려면 1시간 5분 이하로 줄여야 합니다.
    - 자전거: 상위 10%에 진입하려면 6시간 0분 이하로 줄여야 합니다.
    - 달리기: 현재 기록이 평균 이상이며 추가 개선은 필요하지 않습니다.


In [71]:
# Spark DataFrame -> Pandas DataFrame 변환
result_pd = predictions.select("features", "rank_range_index", "prediction", "probability").toPandas()

# 클래스 매핑 (숫자 -> 범위 이름)
class_mapping = {
    0.0: "Bottom 50%",
    1.0: "Top 50%",
    2.0: "Top 25%",
    3.0: "Top 10%"
}

# 매핑 적용
result_pd["Actual Rank"] = result_pd["rank_range_index"].map(class_mapping)
result_pd["Predicted Rank"] = result_pd["prediction"].map(class_mapping)

# 확률 정보 문자열로 포맷
result_pd["Probability"] = result_pd["probability"].apply(
    lambda x: f"Bottom 50%: {x[0]:.2%}, Top 50%: {x[1]:.2%}, Top 25%: {x[2]:.2%}, Top 10%: {x[3]:.2%}"
)

# 필요 없는 컬럼 제거
result_pd = result_pd[["Actual Rank", "Predicted Rank", "Probability"]]

# 결과 출력
print(result_pd.head(10))


AnalysisException: cannot resolve '`rank_range_index`' given input columns: [age_group, bike_seconds, features, gender_encoded, prediction, probability, rawPrediction, run_seconds, swim_seconds];
'Project [features#2439, 'rank_range_index, prediction#2474, probability#2459]
+- Project [gender_encoded#2428, age_group#2429, swim_seconds#2430, bike_seconds#2431, run_seconds#2432, features#2439, rawPrediction#2448, probability#2459, UDF(rawPrediction#2448) AS prediction#2474]
   +- Project [gender_encoded#2428, age_group#2429, swim_seconds#2430, bike_seconds#2431, run_seconds#2432, features#2439, rawPrediction#2448, UDF(rawPrediction#2448) AS probability#2459]
      +- Project [gender_encoded#2428, age_group#2429, swim_seconds#2430, bike_seconds#2431, run_seconds#2432, features#2439, UDF(features#2439) AS rawPrediction#2448]
         +- Project [gender_encoded#2428, age_group#2429, swim_seconds#2430, bike_seconds#2431, run_seconds#2432, UDF(struct(gender_encoded, gender_encoded#2428, age_g

In [69]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

# 변환 함수 정의
def convert_to_ironman_standard(input_data):
    swim_factor = 3.8 / 1.5
    bike_factor = 180 / 40
    run_factor = 42.2 / 10
    converted_data = []
    for row in input_data:
        gender, age, swim, bike, run = row
        converted_data.append([
            gender,
            age,
            swim * swim_factor,
            bike * bike_factor,
            run * run_factor
        ])
    return converted_data

# 1. Spark 세션 및 모델 로드
spark = SparkSession.builder.appName("Ironman Prediction").getOrCreate()

# 2. Assembler 설정
assembler = VectorAssembler(inputCols=["gender_encoded", "age_group", "swim_seconds", "bike_seconds", "run_seconds"], outputCol="features")

# 3. 사용자 데이터 변환
input_data = [
    [1.0, 33.0, 1500.0, 3600.0, 2400.0],
    [0.0, 28.0, 1600.0, 4000.0, 2600.0]
]
converted_data = convert_to_ironman_standard(input_data)
input_df = spark.createDataFrame(converted_data, schema=["gender_encoded", "age_group", "swim_seconds", "bike_seconds", "run_seconds"])
input_features = assembler.transform(input_df)

# 4. 예측 수행
rf_model = ...  # 학습된 RandomForest 모델 로드
predictions = rf_model.transform(input_features)

# 5. 결과 처리 및 출력
result = predictions.select("features", "prediction", "probability").toPandas()
class_mapping = {0.0: "Bottom 50%", 1.0: "Top 50%", 2.0: "Top 25%", 3.0: "Top 10%"}
result["Predicted Rank"] = result["prediction"].map(class_mapping)
result["Probability"] = result["probability"].apply(
    lambda x: f"Bottom 50%: {x[0]:.2%}, Top 50%: {x[1]:.2%}, Top 25%: {x[2]:.2%}, Top 10%: {x[3]:.2%}"
)
print(result[["Predicted Rank", "Probability"]])


AttributeError: 'ellipsis' object has no attribute 'transform'

In [61]:

# 사용자 데이터 예시
input_data = [
    [1.0, 33.0, 7500.0, 16000.0, 14000.0],  # Gender, Age, Swim, Bike, Run
    [0.0, 28.0, 9000.0, 18000.0, 17000.0]
]

# 입력 데이터 변환
input_df = spark.createDataFrame(input_data, schema=["gender_encoded", "age_group", "swim_seconds", "bike_seconds", "run_seconds"])
input_features = assembler.transform(input_df)

# 예측 수행
predictions = rf_model.transform(input_features)

# 결과 출력 포맷
result = predictions.select("features", "prediction", "probability").toPandas()

# 결과 변환
result["Predicted Rank"] = result["prediction"].map(class_mapping)
result["Probability"] = result["probability"].apply(
    lambda x: f"Bottom 50%: {x[0]:.2%}, Top 50%: {x[1]:.2%}, Top 25%: {x[2]:.2%}, Top 10%: {x[3]:.2%}"
)

# 최종 출력
print(result[["Predicted Rank", "Probability"]])


  Predicted Rank                                        Probability
0        Top 50%  Bottom 50%: 28.34%, Top 50%: 54.46%, Top 25%: ...
1     Bottom 50%  Bottom 50%: 97.48%, Top 50%: 2.52%, Top 25%: 0...


In [62]:
# 사용자 입력 데이터
input_data = [[1.0, 33.0, 1608.0, 5975.0, 3091.0]]  # Gender, Age, Swim, Bike, Run in seconds

# 입력 데이터를 Spark DataFrame으로 생성
input_df = spark.createDataFrame(input_data, schema=["gender_encoded", "age_group", "swim_seconds", "bike_seconds", "run_seconds"])

# 피처 벡터 생성
input_features = assembler.transform(input_df)

# 예측 수행
predictions = rf_model.transform(input_features)

# 결과 Pandas로 변환
result_pd = predictions.select("prediction", "probability").toPandas()


In [63]:
# 클래스 매핑
class_mapping = {
    0.0: "Bottom 50%",
    1.0: "Top 50%",
    2.0: "Top 25%",
    3.0: "Top 10%"
}

# 예상 종합 순위 그룹
overall_rank = class_mapping[result_pd["prediction"].iloc[0]]

# 부문별 순위 그룹 (예제: M30-34 그룹)
division_rank = "Top 10%"  # 부문 순위는 추가 데이터가 필요하거나 가정할 수 있음


In [65]:
# 상위 10% 기준 (예시 데이터)
top_10_criteria = {
    "swim_seconds": 3900,  # 1시간 5분
    "bike_seconds": 21600, # 6시간
    "run_seconds": 16200   # 4시간 30분
}

# 개선 방향 계산
improvement_suggestions = []

if input_data[0][2] > top_10_criteria["swim_seconds"]:
    improvement_suggestions.append(f"수영: 상위 10%에 진입하려면 {top_10_criteria['swim_seconds']//3600}시간 {top_10_criteria['swim_seconds']%3600//60}분 이하로 줄여야 합니다.")
if input_data[0][3] > top_10_criteria["bike_seconds"]:
    improvement_suggestions.append(f"자전거: 상위 10%에 진입하려면 {top_10_criteria['bike_seconds']//3600}시간 {top_10_criteria['bike_seconds']%3600//60}분 이하로 줄여야 합니다.")
if input_data[0][4] > top_10_criteria["run_seconds"]:
    improvement_suggestions.append(f"달리기: 상위 10%에 진입하려면 {top_10_criteria['run_seconds']//3600}시간 {top_10_criteria['run_seconds']%3600//60}분 이하로 줄여야 합니다.")
else:
    improvement_suggestions.append("달리기: 현재 기록이 평균 이상이며 추가 개선은 필요하지 않습니다.")


입력 데이터:
- 성별: 남성
- 나이: 33세
- 기록:
    - 수영 (3.8km): 01:10:00 (1시간 10분)
    - 자전거 (180km): 06:20:00 (6시간 20분)
    - 달리기 (42.2km): 04:30:00 (4시간 30분)

모델 예측 결과:
- 예상 종합 순위 그룹: Top 10%
- 예상 부문 순위 그룹 (M30-34): Top 10%

종목별 개선 방향:
    - 달리기: 현재 기록이 평균 이상이며 추가 개선은 필요하지 않습니다.


In [84]:
spark.stop()